In [1]:
import fitz  # PyMuPDF
import pdfplumber
from PIL import Image
import io
import os

def extract_all_from_pdf(pdf_path, output_dir="./extracted_data"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    all_content = {"texts": [], "tables": [], "images": []}

    # 1. PyMuPDF(fitz)로 텍스트와 이미지 추출
    doc = fitz.open(pdf_path)
    # 2. pdfplumber로 테이블 추출 (레이아웃 보존 능력이 뛰어남)
    pdf_plumber = pdfplumber.open(pdf_path)

    for page_num in range(len(doc)):
        print(f"--- {page_num + 1} 페이지 처리 중 ---")

        # [텍스트 추출]
        page = doc.load_page(page_num)
        all_content["texts"].append(page.get_text())

        # [테이블 추출]
        plumber_page = pdf_plumber.pages[page_num]
        table = plumber_page.extract_table()
        if table:
            all_content["tables"].append(table)

        # [이미지 추출]
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            img_filename = f"page{page_num+1}_img{img_index+1}.{image_ext}"
            with open(os.path.join(output_dir, img_filename), "wb") as f:
                f.write(image_bytes)
            all_content["images"].append(img_filename)

    doc.close()
    pdf_plumber.close()
    return all_content

# 실행
content = extract_all_from_pdf("invest/invest.pdf")
print(f"\n추출 완료: 텍스트 {len(content['texts'])}개, 테이블 {len(content['tables'])}개, 이미지 {len(content['images'])}개")

--- 1 페이지 처리 중 ---
--- 2 페이지 처리 중 ---
--- 3 페이지 처리 중 ---

추출 완료: 텍스트 3개, 테이블 0개, 이미지 6개
